# Sample a Gaussian Random Field
This shows how we can sample a Gaussian random variable by FFT methods.  In this excample, the mean is zero, and the covariance is
$$
C = (-d^2/dx^2)^{-\alpha} + \text{Dirichlet Boundary Conditions}
$$
The eigenvalue/eigenfunction pairs are
$$
\lambda_k = \frac{1}{(\pi k)^{2\alpha}}, \quad \varphi_k(x) = \sqrt{2}\sin(\pi k x)
$$
The parameter $\alpha>1/2$ controls the smoothness of the field.  The output is
$$
u(x) = \sum_{k=1}^\infty \sqrt{\lambda_k}\xi_k \varphi_k(x)
$$
where $\xi_k$ are i.i.d. $N(0,1)$.  

In [ ]:
using FFTW
using Plots
using Random
using LaTeXStrings
using Printf

In [ ]:
default(xtickfont=font(14),  ytickfont=font(14), guidefont=font(14), 
    legendfontsize=12, lw=2,ms=8)

In [ ]:
"""
`sample_field` - Sample a mean zero Gaussian random field with the (-Δ)^{-α} covariance operator in dimension one

### Fields
`ξ`   - Vector of N(0,1) values  
`α=1` - Smoothness parameter
"""
function sample_field(ξ; α=1)
    N = length(ξ)
    
    uhat = zeros(ComplexF64,2*N); # preallocate space

    # construct the eigenvalues
    k = 1:N;
    λ = @. 1/(π*k)^(2*α);

    # fill in the nonzero entries
    # NOTE we need to multiply by 2 *N for FFT scaling
    @. uhat[2:N+1] = 2 * N * sqrt(λ) * sqrt(2) * ξ;

    # invert and get the relevant imaginary part
    u = imag.(ifft(uhat))[N+2:end];
    return u
end

In [ ]:
N = 64;
x = LinRange(0,1,N+1)[2:end-1];
Random.seed!(100);
ξ = randn(N);
u = sample_field(ξ, α=1.0)
plot(x, u,label="")
xlabel!(L"$x$")
ylabel!(L"$u$")

In [ ]:
anim = @animate for N in [8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]
    x = LinRange(0,1,N+1)[2:end-1];
    Random.seed!(100);
    ξ = randn(N);
    u = sample_field(ξ, α=1.0)
    plot(x, u,label="")
    xlabel!(L"$x$")
    ylabel!(L"$u$")
    xticks!([0.25, 0.5, 0.75])
    title!(latexstring(@sprintf("\$ N = %d\$", N)))
    xlims!(0,1)
    ylims!(-0.75, 0.75)
end

gif(anim, fps = 1)

In [ ]:
N = 1024;
x = LinRange(0,1,N+1)[2:end-1];
Random.seed!(100);
ξ = randn(N);
u = sample_field(ξ, α=1.0)
plot(x, u,label="Sample 1")
ξ = randn(N);
u = sample_field(ξ, α=1.0)
plot!(x, u,label="Sample 2")
ξ = randn(N);
u = sample_field(ξ, α=1.0)
plot!(x, u,label="Sample 3")
xlabel!(L"$x$")
ylabel!(L"$u$")